# ALTERNATE ML MODEL FOR BENCHMARKING AND FEATURE IMPORTANCE

In [1]:
# Import the dependencies.
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import numpy as np
import re
import tensorflowjs as tfjs

# Import the CRSS connection string.
from config import crss_conn_string

# CONNECT TO THE DATABASE

In [2]:
# Create CSSR engine.
cssr_engine = create_engine(crss_conn_string);

In [3]:
# Connect to server.
cssr_conn = cssr_engine.connect();

In [4]:
# Create accidents, vehicles, people data frame.
avp_df = pd.read_sql("SELECT * FROM public.accident_vehicle_person", cssr_conn)

In [5]:
# Show first 10 rows.
pd.set_option("display.max_columns", None)
avp_df.head()

,casenum,urbancity,ve_total,ve_forms,permvit,num_inj,month,year,day_week,hour,alcohol,max_sev,wrk_zone,lgt_cond,weather,veh_no,numoccs,m_harmname,makename,tow_vehname,trav_speed,deformedname,towedname,speedrelname,vtrafwayname,vspd_lim,bdytyp_imname,mod_year,p_crash1name,per_no,rest_usename,rest_misname,helm_usename,helm_misname,drinkingname,alc_resname,drugsname,hospitalname,locationname,sex_imname,injsev_imname,peralch_imname,seat_imname,age_im
0,201901349715,2,2,2,2,0,3,2019,7,10,2,0,0,1,10,2,1,Motor Vehicle In-Transport,Hyundai,No Trailing Units,999,Functional Damage,Not Towed,Reported as Unknown,"Two-Way, Not Divided",55,"4-door sedan, hardtop",2005,Negotiating a Curve,1,Reported as Unknown,None Used/Not Applicable,Not Applicable,None Used/Not Applicable,No (Alcohol Not Involved),Test Not Given,No (drugs not involved),Not Transported,Occupant of a Motor Vehicle,Female,No Apparent Injury (O),No (Alcohol Not Involved),"Front Seat, Left Side",28
1,201901349853,2,2,2,4,1,3,2019,4,16,9,1,0,1,1,1,1,Motor Vehicle In-Transport,Toyota,No Trailing Units,0,Minor Damage,Not Reported,No,"Two-Way, Not Divided",98,"4-door sedan, hardtop",2011,Stopped in Roadway,1,Shoulder and Lap Belt Used,No Indication of Mis-Use,Not Applicable,None Used/Not Applicable,Not Reported,Test Not Given,Not Reported,Not Transported,Occupant of a Motor Vehicle,Female,No Apparent Injury (O),No (Alcohol Not Involved),"Front Seat, Left Side",60
2,201901350410,1,3,3,5,1,3,2019,6,19,2,2,0,2,10,1,2,Motor Vehicle In-Transport,Toyota,No Trailing Units,35,Disabling Damage,Towed Due to Disabling Damage,No,"Two-Way, Divided, Positive Median Barrier",45,"4-door sedan, hardtop",1998,Changing Lanes,1,Shoulder and Lap Belt Used,No Indication of Mis-Use,Not Applicable,None Used/Not Applicable,No (Alcohol Not Involved),Not Reported,Yes (drugs involved),Not Transported,Occupant of a Motor Vehicle,Male,No Apparent Injury (O),No (Alcohol Not Involved),"Front Seat, Left Side",19
3,201901350410,1,3,3,5,1,3,2019,6,19,2,2,0,2,10,1,2,Motor Vehicle In-Transport,Toyota,No Trailing Units,35,Disabling Damage,Towed Due to Disabling Damage,No,"Two-Way, Divided, Positive Median Barrier",45,"4-door sedan, hardtop",1998,Changing Lanes,2,Not Reported,None Used/Not Applicable,Not Applicable,None Used/Not Applicable,Not Reported,Test Not Given,Not Reported,Not Transported,Occupant of a Motor Vehicle,Not Reported,No Apparent Injury (O),No (Alcohol Not Involved),Not Reported,998
4,201901350410,1,3,3,5,1,3,2019,6,19,2,2,0,2,10,3,1,Motor Vehicle In-Transport,Honda,No Trailing Units,0,Not Reported,"Towed, Unknown Reason",No,"Two-Way, Divided, Positive Median Barrier",45,"4-door sedan, hardtop",2014,Stopped in Roadway,1,Shoulder and Lap Belt Used,No Indication of Mis-Use,Not Applicable,None Used/Not Applicable,No (Alcohol Not Involved),Test Not Given,No (drugs not involved),Not Transported,Occupant of a Motor Vehicle,Male,No Apparent Injury (O),No (Alcohol Not Involved),"Front Seat, Left Side",21


In [6]:
# Write to CSV
avp_df.to_csv('all_gas_no_brakes.csv')

# PREPROCCESSING

In [7]:
# Describe Data
avp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256440 entries, 0 to 256439
Data columns (total 44 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   casenum         256440 non-null  object
 1   urbancity       256440 non-null  int64 
 2   ve_total        256440 non-null  int64 
 3   ve_forms        256440 non-null  int64 
 4   permvit         256440 non-null  int64 
 5   num_inj         256440 non-null  int64 
 6   month           256440 non-null  int64 
 7   year            256440 non-null  int64 
 8   day_week        256440 non-null  int64 
 9   hour            256440 non-null  int64 
 10  alcohol         256440 non-null  int64 
 11  max_sev         256440 non-null  int64 
 12  wrk_zone        256440 non-null  int64 
 13  lgt_cond        256440 non-null  int64 
 14  weather         256440 non-null  int64 
 15  veh_no          256440 non-null  int64 
 16  numoccs         256440 non-null  int64 
 17  m_harmname      256440 non-nu

In [8]:
# Recode Urban City : 0 = urban, 1 = city
avp_df['urbancity'] = avp_df['urbancity'].replace(1,0)
avp_df['urbancity'] = avp_df['urbancity'].replace(2,1)
avp_df['urbancity'].unique()

array([1, 0], dtype=int64)

In [9]:
# Recode Month : 1 = Jan, 12 = Dec
avp_df['month']= avp_df['month'].replace([1,2,3,4,5,6,7,8,9,10,11,12],['Jan','Feb','Mar','Apr','May',
                                                                             'Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
avp_df['month'].unique()

array(['Mar', 'Jan', 'Feb', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)

In [10]:
# Recode Day of the week 1= Sunday, 7 = Saturday
avp_df['day_week']= avp_df['day_week'].replace([1,2,3,4,5,6,7],['Sun','Mon','Tues','Wed','Thus','Fri','Sat'])
avp_df['day_week'].unique()

array(['Sat', 'Wed', 'Fri', 'Thus', 'Mon', 'Sun', 'Tues'], dtype=object)

In [11]:
# Recode HOUR = if unknown, set at 17 (5 pm - most frequent time)
avp_df['hour'] = avp_df['hour'].replace(99,17)
avp_df['hour'].unique()

array([10, 16, 19, 22,  7,  6, 15, 21, 13, 18, 17,  8, 23, 11,  1,  9, 14,
       12, 20,  3,  0,  5,  2,  4], dtype=int64)

In [12]:
# Recode Alcoholinvolvement of the DRIVER 1= alcohol involved, 2,8,9 =  no alcohol
avp_df['alcohol'] = avp_df['alcohol'].replace([1,2,8,9],[1,0,0,0])
avp_df['alcohol'].unique()

array([0, 1], dtype=int64)

In [13]:
# Recode max severity - accident level 0,8,9 = none, 1,2,5 = minor, 3 = serious, 4,6 = fatal
avp_df['max_sev']=avp_df['max_sev'].replace([0,8,9,1,2,5,3,4,6],['none', 'none','none','minor','minor',
                                                                 'minor','serious','fatal','fatal'])
avp_df['max_sev'].unique()

array(['none', 'minor', 'serious', 'fatal'], dtype=object)

In [14]:
# Recode work zone 0 = no (0), 1,2,3,4 = yes (1)
avp_df['wrk_zone'] = avp_df['wrk_zone'].replace([0,1,2,3,4],[0,1,1,1,1])
avp_df['wrk_zone'].unique()

array([0, 1], dtype=int64)

In [15]:
# Recode lighting conditions 1= daylight, 2,3,6 = dark, 4=dawn 5=dusk, 7,8,9 = other
avp_df['lgt_cond']= avp_df['lgt_cond'].replace([1,2,3,6,4,5,7,8,9],['daylight','dark','dark','dark','dawn','dusk',
                                                                     'other','other','other'])
avp_df['lgt_cond'].unique()

array(['daylight', 'dark', 'dawn', 'dusk', 'other'], dtype=object)

In [16]:
# Recode weather 1 = clear 2,3 = rain/sleet/hail, 4,11 = snow, 5 = fog/smoke, 6=windy, 7=blowing dirt, 
# 10=cloudy, 12=freezing rain, 8, 98, 99 = other
avp_df['weather']=avp_df['weather'].replace([1,2,3,4,11,5,6,7,10,12,8,98,99],['clear','rain_sleet','rain_sleet',
                                                                              'snow_blowsnow','snow_blowsnow',
                                                                              'fog_smoke','windy','blowing_dirt','cloudy',
                                                                              'freezing_rain','other','other','other'])
avp_df['weather'].unique()

array(['cloudy', 'clear', 'rain_sleet', 'snow_blowsnow', 'other',
       'fog_smoke', 'windy', 'blowing_dirt', 'freezing_rain'],
      dtype=object)

In [17]:
avp_df['m_harmname'].unique()

array(['Motor Vehicle In-Transport', 'Rollover/Overturn', 'Live Animal',
       'Embankment', 'Pedalcyclist', 'Concrete Traffic Barrier',
       'Parked Motor Vehicle', 'Other Fixed Object',
       'Tree (Standing Only)', 'Ditch', 'Fence', 'Curb', 'Pedestrian',
       'Cable Barrier', 'Mail Box',
       'Motor Vehicle In-Transport Strikes or is Struck by Cargo, Persons or Objects Set-in-Motion from/by Another Motor Vehicle In Transport',
       'Utility Pole/Light Support', 'Fell/Jumped from Vehicle',
       'Post, Pole or Other Supports', 'Guardrail Face', 'Wall',
       'Reported as Unknown', 'Culvert', 'Guardrail End',
       'Non-Motorist on Personal Conveyance', 'Building',
       'Immersion or Partial Immersion', 'Fire Hydrant',
       'Other Object (not fixed)', 'Fire/Explosion',
       'Traffic Sign Support', 'Bridge Rail (Includes parapet)',
       'Bridge Pier or Support', 'Unknown Object Not Fixed',
       'Impact Attenuator/Crash Cushion', 'Bridge Overhead Structure',
     

In [18]:
# Recode most harmful event - vehicle level
avp_df['m_harmname'] = avp_df['m_harmname'].replace([
   'Motor Vehicle In-Transport',
    'Motor Vehicle in Motion Outside the Trafficway',
    'Working Motor Vehicle',
    
    'Parked Motor Vehicle', 
    
    'Pedalcyclist',
    'Pedestrian',
    'Non-Motorist on Personal Conveyance',
    'Ridden Animal or Animal Drawn Conveyance',
    'Live Animal',
    
    'Traffic Sign Support',
    'Utility Pole/Light Support',
    'Bridge Pier or Support',
    'Guardrail End',
    'Post, Pole or Other Supports',
    'Impact Attenuator/Crash Cushion',
    'Fire Hydrant',
    'Other Fixed Object',
    'Unknown Fixed Object',
    'Mail Box',
    'Traffic Signal Support',
    'Bridge Overhead Structure',
    'Building',
    
    'Bridge Rail (Includes parapet)',
    'Curb', 
    'Guardrail Face',
    'Concrete Traffic Barrier',
    'Other Traffic Barrier',
    'Wall', 
    'Cable Barrier',
    'Fence',
    
    'Shrubbery',
    'Tree (Standing Only)',
    'Boulder',
    'Snow Bank',
    
    'Embankment',
    'Ditch', 
    'Ground',
    'Culvert',
    
    'Fire/Explosion',
    
    'Motor Vehicle In-Transport Strikes or is Struck by Cargo, Persons or Objects Set-in-Motion from/by Another Motor Vehicle In Transport',
    'Object That Had Fallen From Motor Vehicle In-Transport',
    'Unknown Object Not Fixed',
    'Thrown or Falling Object',
    'Other Object (not fixed)',
    
    'Immersion or Partial Immersion',
    
    'Reported as Unknown',
    'Harmful Event, Details Not Reported',
    
    'Pavement Surface Irregularity (Ruts, Potholes, Grates, etc.)',
    'Jackknife (harmful to this vehicle)',
    'Other Non-Collision',
    'Cargo/Equipment Loss, Shift, or Damage [harmful]',
    'Rollover/Overturn',
    
    'Injured In Vehicle (Non-Collision)',
    'Fell/Jumped from Vehicle', 
    
     'Railway Vehicle',
    'Road Vehicle on Rails'    
],
[
    'harm_moving_veh',
    'harm_moving_veh',
    'harm_moving_veh',
    
    'harm_parked_veh',
    
    'harm_ped_animal',
    'harm_ped_animal',
    'harm_ped_animal',
    'harm_ped_animal',
    'harm_ped_animal',
    
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    
    'harm_nat_object',
    'harm_nat_object',
    'harm_nat_object',
    'harm_nat_object',
    
    'harm_terrain',
    'harm_terrain',
    'harm_terrain',
    'harm_terrain',
    
    'harm_fire',
    
    'harm_object',
    'harm_object',
    'harm_object',
    'harm_object',
    'harm_object',
    
    'harm_water',
    
    'harm_unknown',
    'harm_unknown',
    
    'harm_lost_control',
    'harm_lost_control',
    'harm_lost_control',
    'harm_lost_control',
    'harm_lost_control',
    
    'harm_injury_fallout',
    'harm_injury_fallout',
    
    'harm_train',
    'harm_train'    
])
avp_df['m_harmname'].unique()

array(['harm_moving_veh', 'harm_lost_control', 'harm_ped_animal',
       'harm_terrain', 'harm_barrier', 'harm_parked_veh',
       'harm_fixed_manmade', 'harm_nat_object', 'harm_object',
       'harm_injury_fallout', 'harm_unknown', 'harm_water', 'harm_fire',
       'harm_train'], dtype=object)

In [19]:
avp_df['makename'].unique()

array(['Hyundai', 'Toyota', 'Honda', 'Chevrolet', 'Volvo', 'GMC', 'Ford',
       'Dodge', 'Jeep / Kaiser-Jeep / Willys- Jeep', 'Lincoln',
       'Nissan/Datsun', 'Mazda', 'KIA', 'Buick / Opel', 'Not Reported',
       'Subaru', 'International Harvester/Navistar', 'Mercedes-Benz',
       'Infiniti', 'Unknown Make', 'Pontiac', 'Volkswagen', 'Mercury',
       'Audi', 'Saturn', 'Kawasaki', 'Kenworth', 'Chrysler', 'Cadillac',
       'Freightliner', 'Acura', 'BMW', 'Other Import', 'Other Make',
       'Isuzu', 'Yamaha', 'Lexus', 'Mitsubishi', 'Smart', 'Porsche',
       'Suzuki', 'Oldsmobile', 'Fiat', 'Harley-Davidson', 'Peterbilt',
       'Other Domestic Manufacturers', 'Triumph', 'Bluebird',
       'White/Autocar White/GMC', 'Gillig', 'Saab', 'Jaguar', 'Mack',
       'Land Rover', 'Victory', 'AM General', 'Scion', 'MCI', 'Plymouth',
       'Alfa Romeo', 'Grumman', 'Ducati', 'Moto-Guzzi', 'Thomas Built',
       'American Motors', 'Daewoo', 'Eagle'], dtype=object)

In [20]:
# Recode vehicle make as make_country
avp_df['make_country'] = avp_df['makename'].replace([
    'Toyota',
    'Honda',
    'Subaru',
    'Nissan/Datsun',
    'Acura',
    'Suzuki',
    'Lexus',
    'Mazda', 
    'Mitsubishi', 
    'Infiniti', 
    'Isuzu',
    'Scion',
    
    'KIA',
    'Daewoo',
    'Hyundai',
    
    'Chevrolet', 
    'Ford',
    'Pontiac',
    'Cadillac',
    'Dodge', 
    'Chrysler', 
    'GMC',
    'Jeep / Kaiser-Jeep / Willys- Jeep', 
    'Buick / Opel',
    'Other Domestic Manufacturers', 
    'Lincoln',
    'Oldsmobile', 
    'Mercury',
    'Plymouth', 
    'Eagle', 
    'American Motors',
    'Saturn',
    
    'Freightliner', 
    'AM General', 
    'International Harvester/Navistar',
    'Peterbilt',
    'Mack',
    'Kenworth',
    'Thomas Built', 
    'Bluebird',
    'White/Autocar White/GMC',
    'Gillig', 
    'MCI',
    'Grumman',
    
    'Mercedes-Benz',
    'Volkswagen',
    'Audi',
    'BMW', 
    'Smart',
    'Porsche',
    
    'Victory',
    
    'Volvo',
    'Saab', 
    
    'Ducati', 
    'Harley-Davidson', 
    'Yamaha', 
    'Kawasaki',
    'Moto-Guzzi',
    
    'Jaguar',
    'Land Rover',
    'Triumph', 
    
    'Alfa Romeo',
    'Fiat',
    
    'Other Import',
    'Other Make',
    'Unknown Make',
    'Not Reported',
],
[
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    
    'make_Korea',
    'make_Korea',
    'make_Korea',
    
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    
    'make_Germany',
    'make_Germany',
    'make_Germany',
    'make_Germany',
    'make_Germany',
    'make_Germany',
    
    'make_China',
    
    'make_Sweden',
    'make_Sweden',
    
    'make_motorcycle',
    'make_motorcycle',
    'make_motorcycle',
    'make_motorcycle',
    'make_motorcycle',
    
    'make_England',
    'make_England',
    'make_England',
    
    'make_Italy',
    'make_Italy',
    
    'make_other',
    'make_other',
    'make_other',
    'make_other',    
])
avp_df['make_country'].unique()

array(['make_Korea', 'make_Japan', 'make_US', 'make_Sweden', 'make_other',
       'make_US_truck', 'make_Germany', 'make_motorcycle', 'make_Italy',
       'make_England', 'make_China'], dtype=object)

In [21]:
# Recode tow_vehname - towing a vehichle 0 = no, 1 = yes
avp_df['tow_vehname']=avp_df['tow_vehname'].replace([
    'No Trailing Units',
    'One Trailing Unit',
    'Unknown',
    'Two Trailing Units',
    'Vehicle Towing Another Motor Vehicle - Non-Fixed Linkage',
    'Three or More Trailing Units',
    'Vehicle Towing Another Motor Vehicle - Fixed Linkage',
    'Yes, Number of Trailing Units Unknown'
],
[0,1,1,1,1,1,1,1])
avp_df['tow_vehname'].unique()

array([0, 1], dtype=int64)

In [22]:
# Determine median speed for imputing for unknown travel Speed
avp_df['trav_speed_temp'] = avp_df['trav_speed']
avp_df['trav_speed_temp'] = avp_df['trav_speed_temp'].replace([997,998,999],[np.NaN,np.NaN,np.NaN])
avp_df['trav_speed_temp'].describe()

count    126350.000000
mean         22.381559
std          22.198566
min           0.000000
25%           0.000000
50%          15.000000
75%          40.000000
max         150.000000
Name: trav_speed_temp, dtype: float64

In [23]:
# Replace unknown travel speed with median travel speed 
avp_df['trav_speed']=avp_df['trav_speed'].replace([997,998,999],[155,23,23])
avp_df['trav_speed'].describe()

count    256440.000000
mean         22.697348
std          15.593666
min           0.000000
25%          15.000000
50%          23.000000
75%          23.000000
max         155.000000
Name: trav_speed, dtype: float64

In [24]:
# Recode deformedname
avp_df['deformedname']=avp_df['deformedname'].replace([
    'Not Reported',
    'Reported as Unknown'
],
[
    'Minor Damage',
    'Minor Damage'
])
avp_df['deformedname'].unique()

array(['Functional Damage', 'Minor Damage', 'Disabling Damage',
       'No Damage'], dtype=object)

In [25]:
# Recode towed Name 0 = no 1 = yes
avp_df['towedname']=avp_df['towedname'].replace([
    'Towed Due to Disabling Damage',
    'Not Towed',
    'Towed Not Due to Disabling Damage',
    'Towed, Unknown Reason',
    'Not Reported',
    'Towed But Not Due to Disabling Damage',
    'Reported as Unknown'
],[1,0,1,1,0,1,0])
avp_df['towedname'].unique()

array([0, 1], dtype=int64)

In [26]:
avp_df['speedrelname'].unique()

array(['Reported as Unknown', 'No', 'Yes, Too Fast for Conditions',
       'Yes, Specifics Unknown', 'Yes, Exceeded Speed Limit',
       'No Driver Present/Unknown if Driver Present', 'Yes, Racing'],
      dtype=object)

In [27]:
# Recode speed related 0 = n 1 = y
avp_df['speedrelname']=avp_df['speedrelname'].replace([
    'No', 
    'Yes, Too Fast for Conditions',
    'Reported as Unknown',
    'Yes, Exceeded Speed Limit',
    'Yes, Specifics Unknown',
    'No Driver Present/Unknown if Driver Present', 
    'Yes, Racing'
],[0,1,0,1,1,0,1])
avp_df['speedrelname'].unique()

array([0, 1], dtype=int64)

In [28]:
avp_df['vtrafwayname'].unique()

array(['Two-Way, Not Divided',
       'Two-Way,  Divided, Positive  Median Barrier', 'Not Reported',
       'Two-Way, Divided, Unprotected Median',
       'Two-Way, Not Divided With a Continuous Left-Turn Lane',
       'Non-Trafficway or Driveway Access', 'One-Way Trafficway',
       'Entrance/Exit Ramp', 'Reported as Unknown'], dtype=object)

In [29]:
# Recode traffic way name 
avp_df['vtrafwayname']=avp_df['vtrafwayname'].replace([
    'Two-Way, Divided, Unprotected Median',
    'Not Reported',
    'Two-Way,  Divided, Positive  Median Barrier',
    'Two-Way, Not Divided',
    'Two-Way, Not Divided With a Continuous Left-Turn Lane',
    'Non-Trafficway or Driveway Access',
    'Entrance/Exit Ramp',
    'One-Way Trafficway', 
    'Reported as Unknown'
],
[
    'Two_way_div_med_nobar',
    'Two_way',
    'Two_way_div_med_bar',
    'Two_way',
    'Two-way',
    'Parking_lot_driveway',
    'Exit_on_ramp',
    'One-way',
    'Two-way'
])
avp_df['vtrafwayname'].unique()

array(['Two_way', 'Two_way_div_med_bar', 'Two_way_div_med_nobar',
       'Two-way', 'Parking_lot_driveway', 'One-way', 'Exit_on_ramp'],
      dtype=object)

In [30]:
avp_df['vspd_lim'].value_counts()

45    47465
35    42468
98    34694
40    24797
55    23671
25    20691
30    15468
65    12710
70    11755
50     7727
0      6416
60     3798
20     1461
15     1445
75      681
10      466
80      303
99      241
5       182
90        1
Name: vspd_lim, dtype: int64

In [31]:
# Recode speed limits (impute with the mode = 45)
avp_df['vspd_lim']=avp_df['vspd_lim'].replace([98,99],[45,45])
avp_df['vspd_lim'].value_counts()

45    82400
35    42468
40    24797
55    23671
25    20691
30    15468
65    12710
70    11755
50     7727
0      6416
60     3798
20     1461
15     1445
75      681
10      466
80      303
5       182
90        1
Name: vspd_lim, dtype: int64

In [32]:
avp_df['bdytyp_imname'].unique()

array(['4-door sedan, hardtop',
       'Two Wheel Motorcycle (excluding motor scooters)',
       'Large utility (ANSI D16.1 Utility Vehicle Categories and Full Size and Large)',
       'Utility station wagon (includes suburban limousines, Suburban, Travellall, Grand Wagoneer)',
       'Light Pickup', '2-door sedan,hardtop,coupe',
       'Other or Unknown automobile type',
       'Compact Utility (Utility Vehicle Categories Small and Midsize)',
       'Station Wagon (excluding van and truck based)',
       'Unknown (pickup style) light conventional truck type',
       '5-door/4-door hatchback', 'Convertible(excludes sun-roof,t-bar)',
       'Truck-tractor (Cab only, or with any number of trailing unit; any weight)',
       'Not Reported', 'Unknown light truck type',
       'Unknown light vehicle type (automobile,utility vehicle, van, or light truck)',
       'School Bus', 'Unknown body type',
       'Single-unit straight truck or Cab-Chassis (GVWR range 19,501 to 26,000 lbs.)',
       '

In [33]:
# Recode body type
avp_df['bdytyp_imname']=avp_df['bdytyp_imname'].replace([
    '4-door sedan, hardtop',
    '5-door/4-door hatchback', 
    'Sedan/Hardtop, number of doors unknown', 
    'Hatchback, number of doors unknown',
    'Auto-based pickup (includes E1 Camino, Caballero, Ranchero, SSR, G8-ST, Subaru Brat, Rabbit Pickup)',
    'Large Limousine-more than four side doors or stretched chassis',
    'Auto-based panel (cargo station wagon, auto-based ambulance or hearse)',
    'Station Wagon (excluding van and truck based)',
    'Other or Unknown automobile type',
    'Unknown body type', 
    'Not Reported',
    
    '2-door sedan,hardtop,coupe',
    '3-door/2-door hatchback',
    '3-door coupe',
    
    'Convertible(excludes sun-roof,t-bar)',
    
    'Light Pickup',
    'Unknown light truck type', 
    'Unknown (pickup style) light conventional truck type',
    'Unknown light vehicle type (automobile,utility vehicle, van, or light truck)',
    'Other light conventional truck type',
    'Compact Utility (Utility Vehicle Categories Small and Midsize)', 
    'Cab Chassis Based (includes Rescue Vehicle, Light Stake, Dump, and Tow Truck)',
    
    'Minivan (Chrysler Town and Country, Caravan, Grand Caravan, Voyager, Voyager, Honda-Odyssey, ...)',
    'Large Van-Includes van-based buses (B150-B350, Sportsman, Royal Maxiwagon, Ram, Tradesman,...)',
    'Van-Based Bus GVWR greater than 10,000 lbs.', 
    'Unknown van type',
    'Step van (GVWR greater than 10,000 lbs.)',
    'Other van type (Hi-Cube Van, Kary)', 
    'Step-van or walk-in van (GVWR less than or equal to 10,000 lbs.)',
    
    'Large utility (ANSI D16.1 Utility Vehicle Categories and Full Size and Large)',
    'Medium/heavy Pickup (GVWR greater than 10,000 lbs.)',
    'Utility Vehicle, Unknown body type', 
    'Utility station wagon (includes suburban limousines, Suburban, Travellall, Grand Wagoneer)',
    'Unknown truck type (light/medium/heavy)',
    
    'Truck-tractor (Cab only, or with any number of trailing unit; any weight)',
    'Single-unit straight truck or Cab-Chassis (GVWR range 19,501 to 26,000 lbs.)',
    'Single-unit straight truck or Cab-Chassis (GVWR unknown)',
    'Unknown medium/heavy truck type', 
    'Single-unit straight truck or Cab-Chassis (GVWR range 10,001 to 19,500 lbs.)',
    'Single-unit straight truck or Cab-Chassis (GVWR greater than 26,000 lbs.)',
    'Unknown if single-unit or combination unit Medium Truck (GVWR range 10,001 lbs. to 26,000 lbs.)',
    'Unknown if single-unit or combination unit Heavy Truck (GVWR greater than 26,000 lbs.)',
    
    'Two Wheel Motorcycle (excluding motor scooters)',
    'Motor Scooter',
    'Moped or motorized bicycle',
    'Unknown motored cycle type',
    'Off-road Motorcycle', 
    'Unenclosed Three Wheel Motorcycle / Unenclosed Autocycle (1 Rear Wheel)',
    'Three-wheel Motorcycle (2 Rear Wheels)',
    'Other motored cycle type (mini-bikes, pocket motorcycles pocket bikes)',
    'Unknown Three Wheel Motorcycle Type',
    
    'School Bus',
    'Transit Bus (City Bus)',
    'Other Bus Type', 
    'Unknown Bus Type',
    'Cross Country/Intercity Bus',
    
    'Construction equipment other than trucks (includes graders)',  
    'Farm equipment other than trucks',
    
    'Medium/heavy truck based motorhome',
    'Medium/Heavy Vehicle Based Motor Home',
    'Camper or motorhome, unknown truck type',
    'Light Vehicle Based Motor Home (chassis mounted)',
    'Light Truck Based Motorhome (Chassis Mounted)',
    
    'Recreational Off-Highway Vehicle',
    'Other vehicle type (includes go-cart, fork-lift, city street sweeper dunes/swamp buggy)',
    'Low Speed Vehicle (LSV) / Neighborhood Electric Vehicle (NEV)',
    'ATV/ATC [All-Terrain Cycle]',
    'Golf Cart',
     
     'Snowmobile'
    
],[
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    
    '2_door_sedan',
    '2_door_sedan',
    '2_door_sedan',
    
    'Convertable',
    
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    
    'Van',
    'Van',
    'Van',
    'Van',
    'Van',
    'Van',
    'Van',
    
    'Large_SUV',
    'Large_SUV',
    'Large_SUV',
    'Large_SUV',
    'Large_SUV',
    
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    
    'Bus',
    'Bus',
    'Bus',
    'Bus',
    'Bus',
    
    'Construction_farm_equip',
    'Construction_farm_equip',
    
    'Motorhome_RV',
    'Motorhome_RV',
    'Motorhome_RV',
    'Motorhome_RV',
    'Motorhome_RV',
    
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    
     'ATV_rec_vehicle',
    
])
avp_df['bdytyp_imname'].unique()


array(['4_door_sedan', 'Motorcylcle_trike', 'Large_SUV',
       'Small_SUV_light_truck', '2_door_sedan', 'Convertable', 'Truck',
       'Bus', 'Van', 'ATV_rec_vehicle', 'Construction_farm_equip',
       'Motorhome_RV'], dtype=object)

In [34]:
avp_df['p_crash1name'].unique()

array(['Negotiating a Curve', 'Stopped in Roadway', 'Changing Lanes',
       'Going Straight', 'Turning Left', 'Decelerating in Road',
       'Turning Right', 'Passing or Overtaking Another Vehicle',
       'Backing Up (other than for Parking Position)', 'Making a U-turn',
       'Unknown', 'Merging', 'Leaving a Parking Position',
       'Starting in Road',
       'No Driver Present / Unknown if Driver Present',
       'Disabled or Parked in Travel lane', 'Other(specify:)',
       'Successful Avoidance Maneuver to a Previous Critical Event',
       'Accelerating in Road', 'Entering a Parking Position'],
      dtype=object)

In [35]:
# Recode prior to crash movement: some not binned. unknowns binned with going straight
avp_df['p_crash1name'] = avp_df['p_crash1name'].replace([
    'Going Straight', 
    'Other(specify:)',
    'Unknown', 
    'No Driver Present / Unknown if Driver Present',
        
    'Decelerating in Road',
    'Stopped in Roadway',
    'Backing Up (other than for Parking Position)',
    
    'Starting in Road',
    'Accelerating in Road', 
    
    'Disabled or Parked in Travel lane', 
    'Entering a Parking Position',
    'Leaving a Parking Position',
     
]
,[
    'Going straight',
    'Going straight',
    'Going straight',
    'Going straight',
    
    'Stopping_backup',
    'Stopping_backup',
    'Stopping_backup',
    
    'Start on road',
    'Start on road',
    
    'Diasbled_parked',
    'Diasbled_parked',
    'Diasbled_parked',
])
avp_df['p_crash1name'].unique()

array(['Negotiating a Curve', 'Stopping_backup', 'Changing Lanes',
       'Going straight', 'Turning Left', 'Turning Right',
       'Passing or Overtaking Another Vehicle', 'Making a U-turn',
       'Merging', 'Diasbled_parked', 'Start on road',
       'Successful Avoidance Maneuver to a Previous Critical Event'],
      dtype=object)

In [36]:
# Recode restrictive device
avp_df['rest_usename'] = avp_df['rest_usename'].replace([
    'Shoulder and Lap Belt Used',
    'Shoulder Belt Only Used', 
    'Other',
    'Restraint Used - Type Unknown',
    'Lap Belt Only Used',
    'Reported as Unknown',
    'Not Reported',
    
    'None Used/Not Applicable', 

    'Child Restraint Type Unknown', 
    'Booster Seat', 
    'Child Restraint System - Forward Facing',
    'Child Restraint System  - Rear Facing',
    'Child Restraint System - Rear Facing',
    
    'Racing-Style Harness Used'
    
   
],[
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    
    'No_seatbelt',
    
    'Child_restraint',
    'Child_restraint',
    'Child_restraint',
    'Child_restraint',
    'Child_restraint',
    
    'Harness'
])

avp_df['rest_usename'].unique()

array(['Seatbelt', 'No_seatbelt', 'Child_restraint', 'Harness'],
      dtype=object)

In [37]:
# Recode retrain misuse 0 = no, 1 = yes
avp_df['rest_misname'] = avp_df['rest_misname'].replace([
    'No Indication of Mis-Use', 
    'None Used/Not Applicable',
    'Yes, Indication of Mis-Use'
],[0,0,1])
avp_df['rest_misname'].unique()

array([0, 1], dtype=int64)

In [38]:
# Recode helmet wear 0= no, 1=yes
avp_df['helm_usename']=avp_df['helm_usename'].replace([
    'Not Applicable',
    'Helmet, Other than DOT-Compliant Motorcycle Helmet',
    'Helmet, Unknown if DOT-Compliant', 
    'No Helmet', 
    'Not Reported',
    'DOT-Compliant Motorcycle Helmet',
    'Reported as Unknown if Helmet Worn'
],[0,1,1,0,0,1,0])
avp_df['helm_usename'].unique()

array([0, 1], dtype=int64)

In [39]:
# Recode helmet misuse 0 = no 1 = yes
avp_df['helm_misname']=avp_df['helm_misname'].replace([
    'None Used/Not Applicable', 
    'No Indication of Mis-Use',
    'Yes, Indication of Mis-Use'
],[0,0,1])
avp_df['helm_misname'].unique()

array([0, 1], dtype=int64)

In [40]:
# Recode police reported drinking in the car 0 = no 1 = yes
avp_df['drinkingname'] = avp_df['drinkingname'].replace([
    'No (Alcohol Not Involved)', 
    'Not Reported', 
    'Reported as Unknown',
    'Yes (Alcohol Involved)'
],[0,0,0,1])
avp_df['drinkingname'].unique()

array([0, 1], dtype=int64)

In [41]:
avp_df['alc_resname'].unique()


array(['Test Not Given', 'Not Reported',
       'AC Test Performed, Results Unknown',
       'Reported as Unknown if Tested', '0.000 % BAC', '0.121 % BAC',
       '0.050 % BAC', '0.076 % BAC', '0.181 % BAC', '0.230 % BAC',
       '0.080 % BAC', 'Positive Reading with No Actual Value',
       '0.210 % BAC', '0.152 % BAC', '0.255 % BAC', '0.125 % BAC',
       '0.075 % BAC', '0.155 % BAC', '0.229 % BAC', '0.190 % BAC',
       '0.160 % BAC', '0.026 % BAC', '0.245 % BAC', '0.123 % BAC',
       '0.090 % BAC', '0.310 % BAC', '0.200 % BAC', '0.126 % BAC',
       '0.361 % BAC', '0.110 % BAC', '0.140 % BAC', '0.130 % BAC',
       '0.008 % BAC', '0.226 % BAC', '0.262 % BAC', '0.378 % BAC',
       '0.184 % BAC', '0.171 % BAC', '0.158 % BAC', '0.028 % BAC',
       '0.220 % BAC', '0.224 % BAC', '0.250 % BAC', '0.041 % BAC',
       '0.197 % BAC', '0.070 % BAC', '0.209 % BAC', '0.205 % BAC',
       '0.300 % BAC', '0.172 % BAC', '0.142 % BAC', '0.177 % BAC',
       '0.270 % BAC', '0.101 % BAC', '0.395 

In [42]:
# Recode is in alcohol percent >=0.08 for the person (not specifically driver) 0=no 1=yes
avp_df['alc_resname_08']=avp_df['alc_resname'].replace([
    
    'Test Not Given', 'Not Reported', 'Reported as Unknown if Tested',
       'AC Test Performed, Results Unknown', '0.220 % BAC', '0.000 % BAC',
       '0.070 % BAC', '0.176 % BAC', '0.200 % BAC', '0.080 % BAC',
       '0.244 % BAC', '0.170 % BAC', '0.020 % BAC', '0.125 % BAC',
       '0.216 % BAC', '0.140 % BAC', '0.090 % BAC', '0.190 % BAC',
       '0.230 % BAC', '0.150 % BAC',
       'Positive Reading with No Actual Value', '0.172 % BAC',
       '0.160 % BAC', '0.187 % BAC', '0.100 % BAC', '0.110 % BAC',
       '0.181 % BAC', '0.157 % BAC', '0.210 % BAC', '0.158 % BAC',
       '0.119 % BAC', '0.180 % BAC', '0.189 % BAC', '0.130 % BAC',
       '0.030 % BAC', '0.208 % BAC', '0.245 % BAC', '0.144 % BAC',
       '0.121 % BAC', '0.280 % BAC', '0.195 % BAC', '0.194 % BAC',
       '0.017 % BAC', '0.011 % BAC', '0.050 % BAC', '0.076 % BAC',
       '0.116 % BAC', '0.060 % BAC', '0.072 % BAC', '0.221 % BAC',
       '0.193 % BAC', '0.296 % BAC', '0.161 % BAC', '0.255 % BAC',
       '0.075 % BAC', '0.155 % BAC', '0.229 % BAC', '0.214 % BAC',
       '0.152 % BAC', '0.250 % BAC', '.94 % or Greater', '0.238 % BAC',
       '0.123 % BAC', '0.310 % BAC', '0.014 % BAC', '0.275 % BAC',
       '0.290 % BAC', '0.188 % BAC', '0.137 % BAC', '0.114 % BAC',
       '0.040 % BAC', '0.212 % BAC', '0.078 % BAC', '0.184 % BAC',
       '0.375 % BAC', '0.178 % BAC', '0.088 % BAC', '0.165 % BAC',
       '0.115 % BAC', '0.209 % BAC', '0.300 % BAC', '0.168 % BAC',
       '0.132 % BAC', '0.260 % BAC', '0.318 % BAC', '0.185 % BAC',
       '0.286 % BAC', '0.173 % BAC', '0.077 % BAC', '0.186 % BAC',
       '0.120 % BAC', '0.167 % BAC', '0.162 % BAC', '0.111 % BAC',
       '0.129 % BAC', '0.287 % BAC', '0.081 % BAC', '0.302 % BAC',
       '0.094 % BAC', '0.240 % BAC', '0.135 % BAC', '0.142 % BAC',
       '0.131 % BAC', '0.148 % BAC', '0.128 % BAC', '0.309 % BAC',
       '0.026 % BAC', '0.159 % BAC', '0.196 % BAC', '0.191 % BAC',
       '0.151 % BAC', '0.270 % BAC', '0.108 % BAC', '0.105 % BAC',
       '0.204 % BAC', '0.016 % BAC', '0.057 % BAC', '0.109 % BAC',
       '0.093 % BAC', '0.113 % BAC', '0.307 % BAC', '0.085 % BAC',
       '0.008 % BAC', '0.164 % BAC', '0.269 % BAC', '0.320 % BAC',
       '0.138 % BAC', '0.146 % BAC', '0.062 % BAC', '0.083 % BAC',
       '0.177 % BAC', '0.247 % BAC', '0.175 % BAC', '0.223 % BAC',
       '0.102 % BAC', '0.028 % BAC', '0.222 % BAC', '0.126 % BAC',
       '0.228 % BAC', '0.336 % BAC', '0.920 % BAC', '0.239 % BAC',
       '0.232 % BAC', '0.201 % BAC', '0.281 % BAC', '0.001 % BAC',
       '0.285 % BAC', '0.010 % BAC', '0.197 % BAC', '0.249 % BAC',
       '0.101 % BAC', '0.361 % BAC', '0.395 % BAC', '0.163 % BAC',
       '0.096 % BAC', '0.382 % BAC', '0.271 % BAC', '0.350 % BAC',
       '0.182 % BAC', '0.047 % BAC', '0.211 % BAC', '0.154 % BAC',
       '0.166 % BAC', '0.053 % BAC', '0.042 % BAC', '0.218 % BAC',
       '0.192 % BAC', '0.215 % BAC', '0.004 % BAC', '0.174 % BAC',
       '0.231 % BAC', '0.226 % BAC', '0.227 % BAC', '0.099 % BAC',
       '0.139 % BAC', '0.082 % BAC', '0.124 % BAC', '0.002 % BAC',
       '0.031 % BAC', '0.272 % BAC', '0.213 % BAC', '0.058 % BAC',
       '0.045 % BAC', '0.262 % BAC', '0.198 % BAC', '0.378 % BAC',
       '0.224 % BAC', '0.041 % BAC', '0.390 % BAC', '0.112 % BAC',
       '0.117 % BAC', '0.205 % BAC', '0.305 % BAC', '0.298 % BAC',
       '0.179 % BAC', '0.289 % BAC', '0.242 % BAC', '0.012 % BAC',
       '0.147 % BAC', '0.243 % BAC', '0.248 % BAC', '0.106 % BAC',
       '0.234 % BAC', '0.141 % BAC', '0.274 % BAC', '0.143 % BAC',
       '0.256 % BAC', '0.145 % BAC', '0.006 % BAC', '0.202 % BAC',
       '0.207 % BAC', '0.054 % BAC', '0.380 % BAC', '0.068 % BAC',
       '0.067 % BAC', '0.384 % BAC', '0.027 % BAC', '0.253 % BAC',
       '0.156 % BAC', '0.292 % BAC', '0.136 % BAC', '0.254 % BAC',
       '0.322 % BAC', '0.206 % BAC', '0.237 % BAC', '0.600 % BAC',
       '0.252 % BAC', '0.095 % BAC', '0.340 % BAC', '0.048 % BAC',
       '0.153 % BAC', '0.294 % BAC', '0.337 % BAC', '0.091 % BAC',
       '0.233 % BAC', '0.241 % BAC', '0.043 % BAC', '0.171 % BAC',
       '0.023 % BAC', '0.334 % BAC', '0.520 % BAC', '0.007 % BAC',
       '0.069 % BAC', '0.059 % BAC', '0.104 % BAC', '0.079 % BAC',
       '0.268 % BAC', '0.013 % BAC', '0.107 % BAC', '0.236 % BAC',
       '0.217 % BAC', '0.066 % BAC', '0.098 % BAC', '0.203 % BAC',
       '0.251 % BAC', '0.003 % BAC', '0.308 % BAC', '0.278 % BAC',
       '0.246 % BAC', '0.064 % BAC', '0.235 % BAC', '0.086 % BAC',
       '0.061 % BAC', '0.259 % BAC', '0.169 % BAC'
],[
   0,0,0,
    0,1,0,
    0,1,1,1,
    1,1,0,1,
    1,1,1,1,
    1,1,
    0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    0,1,1,1,
    1,1,1,1,
    0,0,0,0,
    1,0,0,1,
    1,1,1,1,
    0,1,1,1,
    1,1,1,1,
    1,1,0,1,
    1,1,1,1,
    0,1,0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    1,1,0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    0,1,1,1,
    1,1,1,1,
    1,0,0,1,
    1,1,1,1,
    0,1,1,1,
    1,1,0,1,
    1,1,1,1,
    1,0,1,1,
    1,1,1,1,
    1,1,1,0,
    1,0,1,1,
    1,1,1,1,
    1,1,1,1,
    1,0,1,1,
    1,0,0,1,
    1,1,0,1,
    1,1,1,1,
    1,1,1,0,
    0,1,1,0,
    0,1,1,1,
    1,0,1,1,
    1,1,1,1,
    1,1,1,0,
    1,1,1,1,
    1,1,1,1,
    1,1,0,1,
    1,0,1,0,
    0,1,0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,0,
    1,1,1,1,
    1,1,0,1,
    0,1,1,0,
    0,0,1,0,
    1,0,1,1,
    1,0,1,1,
    1,0,1,1,
    1,0,1,1,
    0,1,1   
])
avp_df['alc_resname_08'].unique()

array([0, 1], dtype=int64)

In [43]:
# Recode drugs involved with the driver in the car person is in 0 = no, 1 = yes
avp_df['drugsname']=avp_df['drugsname'].replace([
    'No (drugs not involved)', 
    'Not Reported', 
    'Reported as Unknown',
    'Yes (drugs involved)'
],[0,0,0,1])
avp_df['drugsname'].unique()

array([0, 1], dtype=int64)

In [44]:
# recode transfer to hospital 0 = no 1 = yes
avp_df['hospitalname']=avp_df['hospitalname'].replace([
    'EMS Ground', 
    'Not Transported', 
    'Other', 
    'EMS Air',
    'EMS Unknown Mode', 
    'Reported as Unknown',
    'Transported  Unknown Source', 
    'Not Reported', 
    'Law Enforcement',
    'Not Transported for Treatment'
],[1,0,1,1,1,0,1,0,1,0])
avp_df['hospitalname'].unique()

array([0, 1], dtype=int64)

In [45]:
avp_df['locationname'].unique()
# NOT USEFUL

array(['Occupant of a Motor Vehicle'], dtype=object)

In [46]:
avp_df['sex_imname'].value_counts()

Male                   135342
Female                 109803
Not Reported             7708
Reported as Unknown      3587
Name: sex_imname, dtype: int64

In [47]:
# Recode sex 0 = female, 1 = male : Assign not reported as male, unknown as female
avp_df['sex_imname']=avp_df['sex_imname'].replace([
    'Female', 
    'Male', 
    'Reported as Unknown', 
    'Not Reported'
],[0,1,0,1])

avp_df['sex_imname'].value_counts()

1    143050
0    113390
Name: sex_imname, dtype: int64

In [48]:
# Recode person level of injury
avp_df['injsev_imname'] = avp_df['injsev_imname'].replace([
    'Suspected Serious Injury (A)', 
    'No Apparent Injury (O)',
    'Possible Injury (C)', 
    'Unknown/Not Reported',
    'Suspected Minor Injury (B)',
    'Fatal Injury (K)',
    'Injured, Severity Unknown', 
    'Died Prior to Crash*'
],[
    'serious',
    'none',
    'minor',
    'none',
    'minor',
    'fatal',
    'minor',
    'fatal'
 ])
avp_df['injsev_imname'].unique()

array(['none', 'minor', 'serious', 'fatal'], dtype=object)

In [49]:
# Recode personal alcohol use - DUPLICATE OF DRINKING
avp_df['peralch_imname']=avp_df['peralch_imname'].replace([
    'No (Alcohol Not Involved)',
    'Yes (Alcohol Involved)'
],[0,1])
avp_df['peralch_imname'].unique()

array([0, 1], dtype=int64)

In [50]:
avp_df['seat_imname'].unique()

array(['Front Seat, Left Side', 'Not Reported', 'Second Seat, Left Side',
       'Second Seat, Right Side', 'Front Seat, Right Side',
       'Front Seat, Middle', 'Reported as Unknown', 'Second Seat, Middle',
       'Other Passenger in enclosed passenger or cargo area',
       'Third Seat, Left Side', 'Third Seat, Middle',
       'Third Seat, Right Side', 'Front Seat, Unknown',
       'Second Seat, Other', 'Second Seat, Unknown',
       'Third Seat, Unknown', 'Sleeper Section of Cab (Truck)',
       'Riding on Exterior of Vehicle', 'Front Seat, Other',
       'Appended to a Motor Vehicle for Motion', 'Trailing Unit',
       'Other Passenger in passenger or cargo area, unknown whether or not enclosed',
       'Other Passenger in unenclosed passenger or cargo area',
       'Fourth Seat, Left Side', 'Third Seat, Other',
       'Fourth Seat, Right Side', 'Fourth Seat, Middle',
       'Fourth Seat, Other'], dtype=object)

In [51]:
avp_df['seat_imname']=avp_df['seat_imname'].replace([
    'Front Seat, Left Side', 
    
    'Front Seat, Right Side',
    'Front Seat, Middle',
    'Front Seat, Unknown',
    'Front Seat, Other', 
    'Not Reported',
    'Reported as Unknown', 
    
    'Second Seat, Left Side', 
    'Second Seat, Middle',
    'Second Seat, Right Side', 
    'Second Seat, Unknown', 
    'Second Seat, Other',
    
    'Third Seat, Left Side',
    'Third Seat, Middle', 
    'Third Seat, Right Side', 
    'Third Seat, Unknown', 
    'Third Seat, Other',
    'Fourth Seat, Left Side',
    'Fourth Seat, Right Side', 
    'Fourth Seat, Middle',
    'Fourth Seat, Other',
    
    'Riding on Exterior of Vehicle', 
    'Appended to a Motor Vehicle for Motion',
    
    'Other Passenger in enclosed passenger or cargo area',
    'Sleeper Section of Cab (Truck)',
    'Other Passenger in passenger or cargo area, unknown whether or not enclosed',
    'Other Passenger in unenclosed passenger or cargo area',
    
    'Trailing Unit',
    
],[
    'Driver',
    
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    
    'Second_row',
    'Second_row',
    'Second_row',
    'Second_row',
    'Second_row',
    
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    
    'Riding_outside',
    'Riding_outside',
    
    'Cargo_area',
    'Cargo_area',
    'Cargo_area',
    'Cargo_area',
    
    'Trailer'    
])
avp_df['seat_imname'].unique()

array(['Driver', 'Front_passenger', 'Second_row', 'Cargo_area',
       'Third_or_4 Row', 'Riding_outside', 'Trailer'], dtype=object)

In [52]:
avp_df['age_im_temp']=avp_df['age_im'].replace([997,998,999],[np.NaN,np.NaN,np.NaN])
avp_df['age_im_temp'].describe()

count    238805.000000
mean         37.247930
std          19.046642
min           0.000000
25%          22.000000
50%          34.000000
75%          51.000000
max         120.000000
Name: age_im_temp, dtype: float64

In [53]:
# Recode age - impute median age = 34 years for missing
avp_df['age_im']=avp_df['age_im'].replace([998,999],[34,34])
avp_df['age_im'].describe()

count    256440.000000
mean         37.024575
std          18.398439
min           0.000000
25%          23.000000
50%          34.000000
75%          50.000000
max         120.000000
Name: age_im, dtype: float64

In [54]:
# Recode Numoccs as single occupant if unknown
avp_df['numoccs']=avp_df['numoccs'].replace(99,1)
avp_df['numoccs'].unique()

array([ 1,  2,  3,  4, 34,  5,  8,  6,  7, 13,  9, 21, 11, 10, 16, 18, 48,
       39, 15, 25, 14, 26, 29, 22, 19, 20, 17, 41, 23, 43, 31, 46, 30, 12,
       37, 50, 57, 36, 45, 24, 58, 65, 33, 77, 32, 54, 28], dtype=int64)

In [55]:
# replace model years unknown with middle of the range years
avp_df['mod_year']=avp_df['mod_year'].replace([9998,9999],[2011,2010])
avp_df['mod_year'].unique()

array([2005, 2011, 1998, 2014, 1985, 2016, 2000, 2007, 1988, 2004, 2006,
       2003, 2008, 2018, 2013, 2017, 2015, 2012, 2010, 2019, 2001, 1993,
       2009, 2002, 1992, 1999, 1995, 2020, 1982, 1997, 1977, 1991, 1996,
       1994, 1967, 1963, 1990, 1984, 1987, 1983, 1986, 1976, 2021, 1980,
       1974, 1981, 1989, 1940, 1975, 1978, 1973, 1970, 1932, 1979, 1953,
       1966, 1950, 1971, 1964, 1972, 1965, 1934, 1955, 1958, 1957, 1968,
       1956, 1961, 1954, 1959, 1969, 1960, 1931], dtype=int64)

In [56]:
# OUTCOME CODING

In [57]:
avp_df['injsev_imname'].unique()

array(['none', 'minor', 'serious', 'fatal'], dtype=object)

In [58]:
# recode injsev 3 ways to reflect severity of X level or worse - 3 different potential outcomes
avp_df['any_inj']=avp_df['injsev_imname'].replace(['none', 'minor', 'serious', 'fatal'],[0,1,1,1])
avp_df['serious_inj']=avp_df['injsev_imname'].replace(['none', 'minor', 'serious', 'fatal'],[0,0,1,1])
avp_df['fatal_inj']=avp_df['injsev_imname'].replace(['none', 'minor', 'serious', 'fatal'],[0,0,0,1])

In [59]:
avp_df['any_inj'].value_counts()

0    185446
1     70994
Name: any_inj, dtype: int64

In [60]:
avp_df['serious_inj'].value_counts()

0    242364
1     14076
Name: serious_inj, dtype: int64

In [61]:
avp_df['fatal_inj'].value_counts()

0    254397
1      2043
Name: fatal_inj, dtype: int64

In [62]:
# Write to CSV SEMI CLEAN - NEEDS SOME FIELDS DROPPED - NOT THE FINAL SET FOR MACHINE LEARNING
avp_df.to_csv('all_gas_no_brakes_semiclean.csv')

# FINAL PROJECT MACHINE LEARNING CODE


In [63]:
# Drop fields not useful for ML
crash_df=avp_df.drop(['casenum','ve_total','ve_forms','permvit','num_inj','veh_no','towedname',
             'per_no','hospitalname','locationname','age_im_temp','trav_speed_temp','alc_resname',
                      'peralch_imname','year','injsev_imname','makename','max_sev'],axis = 1)
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256440 entries, 0 to 256439
Data columns (total 33 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   urbancity       256440 non-null  int64 
 1   month           256440 non-null  object
 2   day_week        256440 non-null  object
 3   hour            256440 non-null  int64 
 4   alcohol         256440 non-null  int64 
 5   wrk_zone        256440 non-null  int64 
 6   lgt_cond        256440 non-null  object
 7   weather         256440 non-null  object
 8   numoccs         256440 non-null  int64 
 9   m_harmname      256440 non-null  object
 10  tow_vehname     256440 non-null  int64 
 11  trav_speed      256440 non-null  int64 
 12  deformedname    256440 non-null  object
 13  speedrelname    256440 non-null  int64 
 14  vtrafwayname    256440 non-null  object
 15  vspd_lim        256440 non-null  int64 
 16  bdytyp_imname   256440 non-null  object
 17  mod_year        256440 non-nu

In [64]:
# PREPROCESS THE DATA

In [65]:
crash_df.columns

Index(['urbancity', 'month', 'day_week', 'hour', 'alcohol', 'wrk_zone',
       'lgt_cond', 'weather', 'numoccs', 'm_harmname', 'tow_vehname',
       'trav_speed', 'deformedname', 'speedrelname', 'vtrafwayname',
       'vspd_lim', 'bdytyp_imname', 'mod_year', 'p_crash1name', 'rest_usename',
       'rest_misname', 'helm_usename', 'helm_misname', 'drinkingname',
       'drugsname', 'sex_imname', 'seat_imname', 'age_im', 'make_country',
       'alc_resname_08', 'any_inj', 'serious_inj', 'fatal_inj'],
      dtype='object')

In [66]:
# Check unique categorical 
crash_df.nunique()

urbancity           2
month              12
day_week            7
hour               24
alcohol             2
wrk_zone            2
lgt_cond            5
weather             9
numoccs            47
m_harmname         14
tow_vehname         2
trav_speed        116
deformedname        4
speedrelname        2
vtrafwayname        7
vspd_lim           18
bdytyp_imname      12
mod_year           73
p_crash1name       12
rest_usename        4
rest_misname        2
helm_usename        2
helm_misname        2
drinkingname        2
drugsname           2
sex_imname          2
seat_imname         7
age_im            109
make_country       11
alc_resname_08      2
any_inj             2
serious_inj         2
fatal_inj           2
dtype: int64

In [67]:
occ_counts = crash_df.numoccs.value_counts()
occ_counts

1     147646
2      60596
3      24723
4      14008
5       6016
6       1974
7        700
8        369
9        112
11        61
10        56
12        36
13        20
14        17
17        13
16        10
19         9
22         6
21         6
18         6
15         5
37         5
33         4
24         4
34         3
31         3
23         3
48         3
41         2
20         2
32         2
25         2
29         2
39         2
58         2
77         1
54         1
65         1
30         1
45         1
36         1
57         1
50         1
46         1
43         1
26         1
28         1
Name: numoccs, dtype: int64

In [68]:
# Bin the number of occupants
#replace_occ_counts = list(occ_counts[occ_counts<120].index)
#for occ in replace_occ_counts:
    #crash_df.numoccs = crash_df.numoccs.replace(occ,'more_than_8')
#crash_df.numoccs.value_counts()

In [69]:
crash_df.vspd_lim.value_counts()

45    82400
35    42468
40    24797
55    23671
25    20691
30    15468
65    12710
70    11755
50     7727
0      6416
60     3798
20     1461
15     1445
75      681
10      466
80      303
5       182
90        1
Name: vspd_lim, dtype: int64

In [70]:
# Combine 90 with 80
crash_df['vspd_lim'] = crash_df['vspd_lim'].replace(90,80)
crash_df.vspd_lim.value_counts()

45    82400
35    42468
40    24797
55    23671
25    20691
30    15468
65    12710
70    11755
50     7727
0      6416
60     3798
20     1461
15     1445
75      681
10      466
80      304
5       182
Name: vspd_lim, dtype: int64

In [71]:
crash_df.bdytyp_imname.value_counts()

4_door_sedan               119347
Small_SUV_light_truck       78855
Large_SUV                   17363
Van                         12869
Truck                        8896
2_door_sedan                 8656
Motorcylcle_trike            7550
Convertable                  1587
Bus                          1014
ATV_rec_vehicle               152
Construction_farm_equip        90
Motorhome_RV                   61
Name: bdytyp_imname, dtype: int64

In [72]:
# Write to CSV CLEAN SET FOR MACHINE LEARNING
crash_df.to_csv('all_gas_no_brakes_clean.csv')

In [73]:
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256440 entries, 0 to 256439
Data columns (total 33 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   urbancity       256440 non-null  int64 
 1   month           256440 non-null  object
 2   day_week        256440 non-null  object
 3   hour            256440 non-null  int64 
 4   alcohol         256440 non-null  int64 
 5   wrk_zone        256440 non-null  int64 
 6   lgt_cond        256440 non-null  object
 7   weather         256440 non-null  object
 8   numoccs         256440 non-null  int64 
 9   m_harmname      256440 non-null  object
 10  tow_vehname     256440 non-null  int64 
 11  trav_speed      256440 non-null  int64 
 12  deformedname    256440 non-null  object
 13  speedrelname    256440 non-null  int64 
 14  vtrafwayname    256440 non-null  object
 15  vspd_lim        256440 non-null  int64 
 16  bdytyp_imname   256440 non-null  object
 17  mod_year        256440 non-nu

# RANDOM FOREST

In [74]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [75]:
# Generate our categorical variable lists
crash_cat = crash_df.dtypes[crash_df.dtypes == "object"].index.tolist()

In [76]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(crash_df[crash_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(crash_cat)
encode_df.head()

,month_Apr,month_Aug,month_Dec,month_Feb,month_Jan,month_Jul,month_Jun,month_Mar,month_May,month_Nov,month_Oct,month_Sep,day_week_Fri,day_week_Mon,day_week_Sat,day_week_Sun,day_week_Thus,day_week_Tues,day_week_Wed,lgt_cond_dark,lgt_cond_dawn,lgt_cond_daylight,lgt_cond_dusk,lgt_cond_other,weather_blowing_dirt,weather_clear,weather_cloudy,weather_fog_smoke,weather_freezing_rain,weather_other,weather_rain_sleet,weather_snow_blowsnow,weather_windy,m_harmname_harm_barrier,m_harmname_harm_fire,m_harmname_harm_fixed_manmade,m_harmname_harm_injury_fallout,m_harmname_harm_lost_control,m_harmname_harm_moving_veh,m_harmname_harm_nat_object,m_harmname_harm_object,m_harmname_harm_parked_veh,m_harmname_harm_ped_animal,m_harmname_harm_terrain,m_harmname_harm_train,m_harmname_harm_unknown,m_harmname_harm_water,deformedname_Disabling Damage,deformedname_Functional Damage,deformedname_Minor Damage,deformedname_No Damage,vtrafwayname_Exit_on_ramp,vtrafwayname_One-way,vtrafwayname_Parking_lot_driveway,vtrafwayname_Two-way,vtrafwayname_Two_way,vtrafwayname_Two_way_div_med_bar,vtrafwayname_Two_way_div_med_nobar,bdytyp_imname_2_door_sedan,bdytyp_imname_4_door_sedan,bdytyp_imname_ATV_rec_vehicle,bdytyp_imname_Bus,bdytyp_imname_Construction_farm_equip,bdytyp_imname_Convertable,bdytyp_imname_Large_SUV,bdytyp_imname_Motorcylcle_trike,bdytyp_imname_Motorhome_RV,bdytyp_imname_Small_SUV_light_truck,bdytyp_imname_Truck,bdytyp_imname_Van,p_crash1name_Changing Lanes,p_crash1name_Diasbled_parked,p_crash1name_Going straight,p_crash1name_Making a U-turn,p_crash1name_Merging,p_crash1name_Negotiating a Curve,p_crash1name_Passing or Overtaking Another Vehicle,p_crash1name_Start on road,p_crash1name_Stopping_backup,p_crash1name_Successful Avoidance Maneuver to a Previous Critical Event,p_crash1name_Turning Left,p_crash1name_Turning Right,rest_usename_Child_restraint,rest_usename_Harness,rest_usename_No_seatbelt,rest_usename_Seatbelt,seat_imname_Cargo_area,seat_imname_Driver,seat_imname_Front_passenger,seat_imname_Riding_outside,seat_imname_Second_row,seat_imname_Third_or_4 Row,seat_imname_Trailer,make_country_make_China,make_country_make_England,make_country_make_Germany,make_country_make_Italy,make_country_make_Japan,make_country_make_Korea,make_country_make_Sweden,make_country_make_US,make_country_make_US_truck,make_country_make_motorcycle,make_country_make_other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,

In [77]:
# Merge one-hot encoded features and drop the originals
crash_df = crash_df.merge(encode_df,left_index=True, right_index=True)
crash_df = crash_df.drop(crash_cat,1)
crash_df.head()

C:\Users\leske\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,urbancity,hour,alcohol,wrk_zone,numoccs,tow_vehname,trav_speed,speedrelname,vspd_lim,mod_year,rest_misname,helm_usename,helm_misname,drinkingname,drugsname,sex_imname,age_im,alc_resname_08,any_inj,serious_inj,fatal_inj,month_Apr,month_Aug,month_Dec,month_Feb,month_Jan,month_Jul,month_Jun,month_Mar,month_May,month_Nov,month_Oct,month_Sep,day_week_Fri,day_week_Mon,day_week_Sat,day_week_Sun,day_week_Thus,day_week_Tues,day_week_Wed,lgt_cond_dark,lgt_cond_dawn,lgt_cond_daylight,lgt_cond_dusk,lgt_cond_other,weather_blowing_dirt,weather_clear,weather_cloudy,weather_fog_smoke,weather_freezing_rain,weather_other,weather_rain_sleet,weather_snow_blowsnow,weather_windy,m_harmname_harm_barrier,m_harmname_harm_fire,m_harmname_harm_fixed_manmade,m_harmname_harm_injury_fallout,m_harmname_harm_lost_control,m_harmname_harm_moving_veh,m_harmname_harm_nat_object,m_harmname_harm_object,m_harmname_harm_parked_veh,m_harmname_harm_ped_animal,m_harmname_harm_terrain,m_harmname_harm_train,m_harmname_harm_unknown,m_harmname_harm_water,deformedname_Disabling Damage,deformedname_Functional Damage,deformedname_Minor Damage,deformedname_No Damage,vtrafwayname_Exit_on_ramp,vtrafwayname_One-way,vtrafwayname_Parking_lot_driveway,vtrafwayname_Two-way,vtrafwayname_Two_way,vtrafwayname_Two_way_div_med_bar,vtrafwayname_Two_way_div_med_nobar,bdytyp_imname_2_door_sedan,bdytyp_imname_4_door_sedan,bdytyp_imname_ATV_rec_vehicle,bdytyp_imname_Bus,bdytyp_imname_Construction_farm_equip,bdytyp_imname_Convertable,bdytyp_imname_Large_SUV,bdytyp_imname_Motorcylcle_trike,bdytyp_imname_Motorhome_RV,bdytyp_imname_Small_SUV_light_truck,bdytyp_imname_Truck,bdytyp_imname_Van,p_crash1name_Changing Lanes,p_crash1name_Diasbled_parked,p_crash1name_Going straight,p_crash1name_Making a U-turn,p_crash1name_Merging,p_crash1name_Negotiating a Curve,p_crash1name_Passing or Overtaking Another Vehicle,p_crash1name_Start on road,p_crash1name_Stopping_backup,p_crash1name_Successful Avoidance Maneuver to a Previous Critical Event,p_crash1name_Turning Left,p_crash1name_Turning Right,rest_usename_Child_restraint,rest_usename_Harness,rest_usename_No_seatbelt,rest_usename_Seatbelt,seat_imname_Cargo_area,seat_imname_Driver,seat_imname_Front_passenger,seat_imname_Riding_outside,seat_imname_Second_row,seat_imname_Third_or_4 Row,seat_imname_Trailer,make_country_make_China,make_country_make_England,make_country_make_Germany,make_country_make_Italy,make_country_make_Japan,make_country_make_Korea,make_country_make_Sweden,make_country_make_US,make_country_make_US_truck,make_country_make_motorcycle,make_country_make_other
0,1,10,0,0,1,0,23,0,55,2005,0,0,0,0,0,0,28,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,16,0,0,1,0,0,0,45,2011,0,0,0,0,0,0,60,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,19,0,0,2,0,35,0,45,1998,0,0,0,0,1,1,19,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,19,0,0,2,

#  MODEL ANY INJURY (any_inj : remove serious_inj and fatal_inj as variables)

In [78]:
# Split our preprocessed data into our features and target arrays
y = crash_df["any_inj"]
X = crash_df.drop(["any_inj","serious_inj","fatal_inj"],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\leske\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256440 entries, 0 to 256439
Columns: 125 entries, urbancity to make_country_make_other
dtypes: float64(104), int64(21)
memory usage: 244.6 MB


In [80]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [81]:
# RANDOM FOREST
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators = 128, random_state = 78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, y_pred)
print(f' Random Forest Predictive accuracy: {accuracy_score(y_test, y_pred):.3f}')

 Random Forest Predictive accuracy: 0.798


In [82]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [83]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,43094,3143
Actual 1,9817,8056


Accuracy Score : 0.7978474496958353
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.93      0.87     46237
           1       0.72      0.45      0.55     17873

    accuracy                           0.80     64110
   macro avg       0.77      0.69      0.71     64110
weighted avg       0.79      0.80      0.78     64110



In [84]:
# Calculate feature importance in the Random Forest model.
feature_importances = rf_model.feature_importances_
feature_importances

array([1.21130448e-02, 6.13705353e-02, 5.71708626e-03, 2.75824008e-03,
       2.53947201e-02, 2.03162177e-03, 3.72657017e-02, 5.55018488e-03,
       4.47749837e-02, 6.41293963e-02, 1.86319075e-03, 8.14895828e-03,
       9.69881208e-07, 2.03735496e-03, 1.62025961e-03, 1.25916188e-02,
       8.30182184e-02, 6.09925810e-04, 6.47656852e-03, 8.72498552e-03,
       7.96791242e-03, 7.22218391e-03, 7.52226430e-03, 8.54135901e-03,
       8.19876336e-03, 7.25871222e-03, 7.60814506e-03, 8.37594915e-03,
       9.12467540e-03, 8.63251607e-03, 1.12369481e-02, 1.00322915e-02,
       1.02080966e-02, 9.06620797e-03, 1.06060162e-02, 1.03399171e-02,
       1.07046960e-02, 8.28248730e-03, 1.76171305e-03, 8.75981042e-03,
       2.82845003e-03, 7.80953970e-04, 2.98755585e-05, 1.22345527e-02,
       8.98863205e-03, 7.70546297e-04, 1.04216827e-04, 5.41506897e-03,
       7.23326610e-03, 2.08038788e-03, 2.43163444e-04, 2.53185977e-03,
       2.14690171e-04, 2.70890754e-03, 2.47906845e-04, 1.05151281e-02,
      

In [85]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08301821844656618, 'age_im'),
 (0.06862984485317483, 'deformedname_Disabling Damage'),
 (0.06412939628955176, 'mod_year'),
 (0.06137053531825922, 'hour'),
 (0.044774983697251254, 'vspd_lim'),
 (0.03726570174176527, 'trav_speed'),
 (0.029653349110693474, 'rest_usename_No_seatbelt'),
 (0.025699763864597703, 'deformedname_Minor Damage'),
 (0.025394720119561936, 'numoccs'),
 (0.017095979597869976, 'bdytyp_imname_Motorcylcle_trike'),
 (0.012695453491333027, 'rest_usename_Seatbelt'),
 (0.012591618848761253, 'sex_imname'),
 (0.012297703281884965, 'vtrafwayname_Two_way'),
 (0.012234552716051537, 'weather_clear'),
 (0.012168156025190356, 'deformedname_Functional Damage'),
 (0.01211304484478625, 'urbancity'),
 (0.011236948072172008, 'day_week_Fri'),
 (0.010842381139627296, 'make_country_make_Japan'),
 (0.010704696022872754, 'day_week_Wed'),
 (0.010662636941824379, 'make_country_make_US'),
 (0.01060601616418222, 'day_week_Thus'),
 (0.010515128127249168, 'm_harmname_harm_lost_control'),
 (0.01

# MODEL SERIOUS INJURY

In [86]:
# Split our preprocessed data into our features and target arrays
y = crash_df["serious_inj"]
X = crash_df.drop(["any_inj","serious_inj","fatal_inj"],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\leske\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [87]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [88]:
# RANDOM FOREST
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators = 128, random_state = 78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, y_pred)
print(f' Random Forest Predictive accuracy: {accuracy_score(y_test, y_pred):.3f}')

 Random Forest Predictive accuracy: 0.949


In [89]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [90]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,60262,334
Actual 1,2941,573


Accuracy Score : 0.9489159257526127
Classification Report
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     60596
           1       0.63      0.16      0.26      3514

    accuracy                           0.95     64110
   macro avg       0.79      0.58      0.62     64110
weighted avg       0.94      0.95      0.93     64110



In [91]:
# Calculate feature importance in the Random Forest model.
feature_importances = rf_model.feature_importances_
feature_importances

array([1.36174856e-02, 6.18299870e-02, 1.00003872e-02, 2.38564669e-03,
       2.42458540e-02, 1.49680900e-03, 4.33999643e-02, 8.78342756e-03,
       4.64606807e-02, 6.81269993e-02, 1.37469897e-03, 5.07157042e-03,
       1.19516103e-04, 5.73643424e-03, 4.37501462e-03, 1.71278570e-02,
       8.82532011e-02, 1.37483350e-03, 6.77596968e-03, 9.33375527e-03,
       7.40101421e-03, 6.86375316e-03, 7.30205267e-03, 8.76623559e-03,
       8.55998781e-03, 6.99907975e-03, 7.89210335e-03, 8.20714340e-03,
       8.84509680e-03, 8.91064049e-03, 1.07749812e-02, 9.56858583e-03,
       1.10554246e-02, 1.01596537e-02, 1.01129143e-02, 1.02078706e-02,
       9.84183112e-03, 7.69203168e-03, 1.91070324e-03, 7.86748312e-03,
       2.84665055e-03, 6.04231391e-04, 8.15786821e-07, 1.22433015e-02,
       9.04581380e-03, 1.14807475e-03, 7.02757711e-05, 5.45329403e-03,
       6.58224011e-03, 1.77293456e-03, 2.40486544e-04, 3.70129471e-03,
       1.34819227e-03, 4.19942648e-03, 9.95438513e-04, 1.13110897e-02,
      

In [92]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08825320114848188, 'age_im'),
 (0.06812699930213863, 'mod_year'),
 (0.061829987022521835, 'hour'),
 (0.04646068071523612, 'vspd_lim'),
 (0.04339996434186447, 'trav_speed'),
 (0.04062794665716749, 'rest_usename_No_seatbelt'),
 (0.03225338918674569, 'deformedname_Disabling Damage'),
 (0.024245853983212627, 'numoccs'),
 (0.01778103578792181, 'rest_usename_Seatbelt'),
 (0.01712785699895627, 'sex_imname'),
 (0.013617485571437483, 'urbancity'),
 (0.013453171756032516, 'make_country_make_Japan'),
 (0.0125981361528797, 'vtrafwayname_Two_way'),
 (0.012279720015458975, 'seat_imname_Driver'),
 (0.012243301457146387, 'weather_clear'),
 (0.012233588378770805, 'make_country_make_US'),
 (0.011444592162916873, 'bdytyp_imname_4_door_sedan'),
 (0.011316235902169973, 'deformedname_Minor Damage'),
 (0.011311089730022667, 'm_harmname_harm_lost_control'),
 (0.011055424611972637, 'day_week_Sat'),
 (0.010774981157687212, 'day_week_Fri'),
 (0.010672229510740486, 'seat_imname_Front_passenger'),
 (0.01058326

# MODEL FATAL ACCIDENTS

In [93]:
# Split our preprocessed data into our features and target arrays
y = crash_df["fatal_inj"]
X = crash_df.drop(["any_inj","serious_inj","fatal_inj"],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\leske\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [94]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [95]:
# RANDOM FOREST
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators = 128, random_state = 78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, y_pred)
print(f' Random Forest Predictive accuracy: {accuracy_score(y_test, y_pred):.3f}')

 Random Forest Predictive accuracy: 0.992


In [96]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [97]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,63603,11
Actual 1,490,6


Accuracy Score : 0.9921853065044455
Classification Report
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     63614
           1       0.35      0.01      0.02       496

    accuracy                           0.99     64110
   macro avg       0.67      0.51      0.51     64110
weighted avg       0.99      0.99      0.99     64110



In [98]:
# Calculate feature importance in the Random Forest model.
feature_importances = rf_model.feature_importances_
feature_importances

array([1.50273884e-02, 6.32971130e-02, 8.01766098e-03, 2.40002226e-03,
       2.22947092e-02, 1.38545173e-03, 4.84802199e-02, 1.09144050e-02,
       4.85467713e-02, 7.46571190e-02, 9.28157022e-04, 4.42355668e-03,
       4.57397277e-04, 6.22456995e-03, 6.17329605e-03, 1.77986214e-02,
       1.00210767e-01, 5.67905140e-03, 6.35922576e-03, 9.40876046e-03,
       8.37166708e-03, 7.80170959e-03, 8.11739550e-03, 8.91061890e-03,
       9.39094678e-03, 6.50952144e-03, 8.10733107e-03, 8.71544101e-03,
       9.46663874e-03, 9.35471460e-03, 1.09805971e-02, 1.06933440e-02,
       1.14590305e-02, 1.17247326e-02, 1.05629872e-02, 1.05787099e-02,
       1.09322021e-02, 9.69407398e-03, 2.68956883e-03, 1.00301525e-02,
       3.40906096e-03, 1.20532050e-03, 0.00000000e+00, 1.20367410e-02,
       1.01082086e-02, 1.91217501e-03, 1.78683350e-05, 4.11134975e-03,
       6.57641412e-03, 2.05651620e-03, 3.27621443e-04, 4.34039967e-03,
       6.75831080e-03, 4.88049378e-03, 1.02497164e-03, 8.08958230e-03,
      

In [99]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.10021076658823637, 'age_im'),
 (0.074657118973107, 'mod_year'),
 (0.06329711301148482, 'hour'),
 (0.04854677131321004, 'vspd_lim'),
 (0.0484802199464844, 'trav_speed'),
 (0.02229470922054691, 'numoccs'),
 (0.0195391122692017, 'rest_usename_No_seatbelt'),
 (0.017798621382977338, 'sex_imname'),
 (0.015027388417308054, 'urbancity'),
 (0.014638333244589961, 'make_country_make_Japan'),
 (0.013843727422186291, 'vtrafwayname_Two_way'),
 (0.01328839370089433, 'bdytyp_imname_4_door_sedan'),
 (0.012988374663041743, 'make_country_make_US'),
 (0.012299815709196613, 'bdytyp_imname_Small_SUV_light_truck'),
 (0.012180266653245047, 'p_crash1name_Going straight'),
 (0.012036740957778692, 'weather_clear'),
 (0.012022260661944513, 'seat_imname_Driver'),
 (0.01172473261946407, 'day_week_Sun'),
 (0.011459030454058088, 'day_week_Sat'),
 (0.011341749870356308, 'deformedname_Disabling Damage'),
 (0.01098059710252537, 'day_week_Fri'),
 (0.0109322020872049, 'day_week_Wed'),
 (0.01091440495774871, 'speedreln